In [1]:
!pip install transformers peft accelerate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.6 MB/s eta 0:00:00


In [3]:
small_output_dir = "/content/drive/MyDrive/phi3p5_lora_styleA_small"
medium_output_dir = "/content/drive/MyDrive/phi3p5_lora_styleA_medium"
large_output_dir = "/content/drive/MyDrive/phi3p5_lora_styleA_large"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

In [7]:
# Model ID used during training
model_id = "microsoft/phi-3.5-mini-instruct"

# Paths to the saved models in Google Drive
small_output_dir = "/content/drive/MyDrive/phi3p5_lora_styleA_small"
medium_output_dir = "/content/drive/MyDrive/phi3p5_lora_styleA_medium"
large_output_dir = "/content/drive/MyDrive/phi3p5_lora_styleA_large"


In [8]:
# Configure quantization for 4-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,  # Ensure it matches your training setup
)

def load_lora_model(output_dir):
    """Function to load base model and apply LoRA adapters"""
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto"  # Auto-allocate to available device
    )
    model = PeftModel.from_pretrained(base_model, output_dir)
    model.eval()
    return model

print("Loading models...")

# Load the models
model_small = load_lora_model(small_output_dir)
model_medium = load_lora_model(medium_output_dir)
model_large = load_lora_model(large_output_dir)

print("Models loaded successfully!")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


Loading models...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Models loaded successfully!


tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [9]:
test_descriptions = [
    "Modern condo in downtown with rooftop pool and gym access",
    "Cozy treehouse nestled in the forest with hiking trails nearby",
    "Beachfront villa with private dock and sunset views",
    "Spacious city loft near trendy cafes and nightlife",
    "Rustic cabin surrounded by wildlife, perfect for digital detox"
]

In [10]:
def generate_title(model, description, max_new_tokens=20):
    """Generate Airbnb listing title based on description"""
    prompt = f"""
Generate an appealing Airbnb listing title based on the description below.

Description:
{description}

Title:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True
        )

    # Decode the generated text and extract the title portion
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if "Title:" in generated_text:
        generated_text = generated_text.split("Title:")[-1].strip()

    return generated_text


In [11]:
print("\n===== GENERATED TITLES =====\n")
for i, desc in enumerate(test_descriptions, start=1):
    print(f"=== Example {i} ===")
    print("Listing description:", desc)

    title_small = generate_title(model_small, desc)
    print("[Small dataset title]:", title_small)

    title_medium = generate_title(model_medium, desc)
    print("[Medium dataset title]:", title_medium)

    title_large = generate_title(model_large, desc)
    print("[Large dataset title]:", title_large)

    print("\n" + "="*50 + "\n")



===== GENERATED TITLES =====

=== Example 1 ===
Listing description: Modern condo in downtown with rooftop pool and gym access


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


[Small dataset title]: Modern condo in the centre of London

Description:
This modern condo is located
[Medium dataset title]: Stylish new condo with rooftop pool & gym access in downtown
[Large dataset title]: 2 Bed 2 Bath Studio Overground 10 min from Euston Station & 5


=== Example 2 ===
Listing description: Cozy treehouse nestled in the forest with hiking trails nearby
[Small dataset title]: Treehouse in the forest with hiking trails

Description:
This treehouse is
[Medium dataset title]: Treehouse in Forest with Hiking Trails - Cozy & Private - St Louis - Missouri
[Large dataset title]: Beachy Treehouse in Forest with Hiking Trails Nearby! Guest access Gu


=== Example 3 ===
Listing description: Beachfront villa with private dock and sunset views
[Small dataset title]: Beachfront villa with private dock and sunset views

Description:
This lovely
[Medium dataset title]: Nice beachfront villa, private dock and sunset views close to the beach and other attractions
[Large dataset tit